In [ ]:
#!/usr/bin/env python

import rospy
import actionlib
import actionlib.msg
import assignment_2_2022.msg
import sys, select, time, math
import jupyros as jr
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

from std_srvs.srv import *
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from sensor_msgs.msg import LaserScan
from assignment_2_2022.msg import position_velocity
from assignment_2_2022.srv import goals, goalsRequest, goalsResponse
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from IPython.display import display
from matplotlib.animation import FuncAnimation

x_val, y_val = [], []
X_goal_canceled, Y_goal_canceled = [], []

%matplotlib widget

goal = None
Client = None
Pub = None
PosVel = position_velocity()
Position = None
last_time_pub_odom = 0
last_time_pub_laser = 0

In [ ]:
def send_button_click(b): #Call 'action_client' function
    global goal
    global Client

    goal = assignment_2_2022.msg.PlanningGoal() #Create the goal for the robot
    goal.target_pose.pose.position.x = DestinationXwid.value
    goal.target_pose.pose.position.y = DestinationYwid.value

    Client.send_goal(goal) #Send goal to the server

In [ ]:
def cancel_button_click(b): #Cancel the goal
    global goal
    global Client
    global X_goal_canceled, Y_goal_canceled
    
    Client.cancel_goal()
    X_goal_canceled.append(goal.target_pose.pose.position.x)
    Y_goal_canceled.append(goal.target_pose.pose.position.y)
    goal = None

In [ ]:
DestinationXwid = widgets.FloatText(description='Future X:')
DestinationYwid = widgets.FloatText(description='Future Y:')

cancel_goal_button = widgets.Button(description="Cancel goal", style=ButtonStyle(button_color='Red'))

cancel_goal_button.on_click(cancel_button_click)

send_goal_button = widgets.Button(description = 'Send goal', style=ButtonStyle(button_color='GreenYellow'))

send_goal_button.on_click(send_button_click)

In [ ]:
def PublishValues(msg):
    global Pub
    global PosVel
    global last_time_pub_odom
    global Position
    global x_val, y_val
    
    Position = msg.pose.pose.position #Get the position
    Velocity = msg.twist.twist.linear #Get the twist
    
    PosVel.CurrentX=Position.x
    PosVel.CurrentY=Position.y
    PosVel.VelX=Velocity.x
    PosVel.VelY=Velocity.y
    
    x_val.append(Position.x)
    y_val.append(Position.y)
    
    Pub.publish(PosVel) #Publish the custom message
    
    current_time = time.time() * 1000  #This part is necessary to see costantly the robot position every 100ms
    if current_time - last_time_pub_odom > 500:
        print("\rRobot position: x={}, y={}".format(PosVel.CurrentX, PosVel.CurrentY), end='')
        last_time_pub_odom = current_time

In [ ]:
def callback_laser(msg):
    global last_time_pub_laser
    
    #Only consider obstacles in a 180° field of view in front of the robot
    index_start = len(msg.ranges) // 2 - (len(msg.ranges) // 4)
    index_end = len(msg.ranges) // 2 + (len(msg.ranges) // 4)
    ranges = msg.ranges[index_start:index_end]
    min_distance = round(min(ranges),1)
    
    
    current_time = time.time() * 1000 #Print the distance every 100 ms
    if current_time - last_time_pub_laser > 100:
        print("\rDistance from the closest obstacle: {}".format(min_distance), end='')
        last_time_pub_laser = current_time

In [ ]:
class PositionVisualizer: #Animation class used to plot the robot's position and goal's position
    def __init__(self):
        # Init function
        self.fig, self.ax = plt.subplots()
        # Settings for robot's position plot
        self.ln, = plt.plot([], [], 'bo', label = 'Robot position')
        # Settings for target's position plot
        self.goal_ln, = plt.plot([], [], 'r*', markersize = 10, label = 'Goal position')
        self.goal_canceled_ln, = plt.plot([], [], 'D', markersize = 10, label = 'Goals canceled')
        
        
    
    def plot_init(self):
        # Set axis limits
        self.ax.set_xlim(10, -10)
        self.ax.set_ylim(10, -10)
        # Set the grid
        self.ax.grid(True, color = 'gainsboro')
        # Set the title
        self.ax.set_title('Robot position and goals')
        # Set the legend
        self.ax.legend(loc = 'upper right')
        return self.ln, self.goal_ln, self.goal_canceled_ln
    
    
    def update_plot(self, frame):
        # Update data
        global x_val, y_val
        self.ln.set_data(x_val, y_val)
        global goal
        global X_goal_canceled, Y_goal_canceled

        if goal is not None:
            self.goal_ln.set_data(goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        else:
            self.goal_ln.set_data([], [])
            
        if not X_goal_canceled:
            self.goal_canceled_ln.set_data(X_goal_canceled, Y_goal_canceled)
        else:
            self.goal_canceled_ln.set_data([], [])
            
        return self.ln, self.goal_ln    

In [ ]:
rospy.init_node('input')

global Pub
Pub=rospy.Publisher("/pos_vel",position_velocity,queue_size=1) #Send a message with velocity and position

global Client
Client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction) #Create the action client
Client.wait_for_server()

In [ ]:
jr.subscribe('/scan', LaserScan, callback_laser)

In [ ]:
jr.subscribe("/odom",Odometry,PublishValues)

In [ ]:
display(widgets.HBox([DestinationXwid, DestinationYwid]))

In [ ]:
display(widgets.HBox([send_goal_button, cancel_goal_button]))

In [ ]:
# Create the visualizer object
position_visualizer = PositionVisualizer()

# Plot
position_animation = FuncAnimation(
    position_visualizer.fig,
    position_visualizer.update_plot,
    init_func = position_visualizer.plot_init,
    cache_frame_data = False)
plt.show(block = True)